# STAT 3612 Final Project

**Interpretable Machine Learning Model On HELOC Data**


---

Choi Suhyun (3035554658)

CHONG Inbum (3035553355)

Kathleen Low Zi Yi (3035549017)

Wong Beatrice Chi Kan (3035374436)

Kim Sunghyun (3035603526)

---
**Abstract:**<br>
Abstract here


# Table of Contents

1.   Introduction
2.   Preparation
3.   Exploratory Data Analysis
4.   Feature Engineering
5.   ...
6.   ...
7.   ...



# Introduction

Introduction paragraph here


---

Variable Definition:

# Preparation

a. Import Packages<br>
b. Load and preview the dataset<br>
c. Encoding 'RiskFlag'<br>
d. Split the dataset into train and test data set


## a. Import Packages

In [ ]:
!pip install -U pygam

     |████████████████████████████████| 1.8MB 5.4MB/s 


In [ ]:
#Any packages using, put here
import pandas as pd
import numpy as np
import sklearn
!pip install -U pygam
!pip install scorecardpy


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

from pygam import LogisticGAM, s, f

import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
import seaborn as sns
import scorecardpy as sc

Requirement already up-to-date: pygam in /usr/local/lib/python3.6/dist-packages (0.8.0)
     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for scorecardpy: filename=scorecardpy-0.1.9.2-cp36-none-any.whl size=57827 sha256=c26e2b5a943e06804ed57f54ae32dc431f9d82473836c4163d27b294955506f7
  Stored in directory: /root/.cache/pip/wheels/1d/66/14/23aeb4e1128278f5880580e2fdd10e956de112a5d56a3a043e
Successfully built scorecardpy


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## b. Load and Preview the dataset

In [ ]:
# Heloc Data
from google.colab import files
uploaded = files.upload()

Saving HelocData.csv to HelocData.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['HelocData.csv']))
# Dataset is now stored in a Pandas Dataframe
print(df)

      RiskFlag  x1   x2  x3   x4  x5  x6  ...  x17  x18  x19  x20  x21  x22  x23
0          Bad  75  169   2   59  21   0  ...    4   43  112    4    6    0   83
1          Bad  66  502   4  145  34   0  ...    3   80   53   17    3   12   83
2         Good  69  338   2   62  22   0  ...    4   25  100    3    2    1   45
3         Good  75  422   1   91  55   0  ...    4    2   11   12    2    1   57
4          Bad  63  242   2   68  25   0  ...    3   73   -8   12    1    5   87
...        ...  ..  ...  ..  ...  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...
10454     Good  89  425  19  186  16   0  ...    0    0   -8    2   -8    0   50
10455      Bad  68   93  16   59  10   0  ...    1   51   33    1    2    1   50
10456     Good  87  325   6  102  27   0  ...    1    1   72    7    5    0   71
10457     Good  75  413   9  112  33   0  ...    0   59   49    5    2    1   80
10458     Good  81  220   3   86  44   0  ...    2    0   -8    1    1    0   14

[10459 rows x 24 columns]


In [ ]:
# Heloc Data Dic
uploaded_2 = files.upload()

Saving HelocDataDict.xlsx to HelocDataDict.xlsx


In [26]:
#Creating dataframe with description of variables
data_dict = pd.read_excel(io.BytesIO(uploaded_2['HelocDataDict.xlsx']))
data_dict

,Variable Names,Description
0,RiskFlag,Paid as negotiated flag (12-36 Months). String...
1,x1,Consolidated version of risk markers
2,x2,Months Since Oldest Trade Open
3,x3,Months Since Most Recent Trade Open
4,x4,Average Months in File
5,x5,Number Satisfactory Trades
6,x6,Number Trades 60+ Ever
7,x7,Number Trades 90+ Ever
8,x8,Percent Trades Never Delinquent
9,x9,Months Since Most Recent Delinquency


In [27]:
# Storing feature description in "var_names"

var_names = data_dict['Description'][1:].str.split('(\.| \()', expand=True).iloc[:,0]
var_names.name = 'Description'
var_names

1                 Consolidated version of risk markers
2                       Months Since Oldest Trade Open
3                  Months Since Most Recent Trade Open
4                               Average Months in File
5                           Number Satisfactory Trades
6                               Number Trades 60+ Ever
7                               Number Trades 90+ Ever
8                      Percent Trades Never Delinquent
9                 Months Since Most Recent Delinquency
10              Max Delq/Public Records Last 12 Months
11                                Max Delinquency Ever
12                              Number of Total Trades
13             Number of Trades Open in Last 12 Months
14                          Percent Installment Trades
15             Months Since Most Recent Inq excl 7days
16                         Number of Inq Last 6 Months
17              Number of Inq Last 6 Months excl 7days
18                       Net Fraction Revolving Burden
19        

## c. Encoding 'RiskFlag'

In [ ]:
encode = LabelEncoder()
df['RiskFlag'] = encode.fit_transform(df['RiskFlag'])
print(df)

## d. Spliting the data into training and test set

In [ ]:
np.random.seed(12345678)  #Was it our group number
df_train, df_test = train_test_split(df, test_size=0.2)
df_train.head()

# Exploratory Data Analysis

a. General dataset information (dtype, no. row and col, etc. + Visualizations for noticeable characteristics)<br>
b. Missing values (With extended ellaboration on each feature info)<br>
c. Imputation (Perhaps better imputation method?)<br>
d. Correlation analysis (Potential colinearity too)<br>
e. Analysis on individual features (Refer to ipynb sent by Kelly)

## a. General dataset information

In [ ]:
df_train.describe()

## b. Missing values

In [ ]:
for i in df_train.columns:
  df_train[i] = df_train[i].replace([-7,-8,-9], np.nan)

df_train.head

## c. Imputation

In [ ]:
#Mean
SI = SimpleImputer(missing_values=np.nan)
SI.fit(df_train.iloc[:, 1:])
df_train.iloc[:, 1:] = SI.transform(df_train.iloc[:, 1:])

nans = df_train.iloc[:, 1:].isnull().sum().sum()
df_train.head

## d. Correlation analysis

## e. Analysis on individual features

# Feature Engineering

Feature Selection<br>
Roughly 10 features will be selected based on three standards:

a. Model-free feature selection <br>
Univariate feature selection

b. Model based feature selection <br>
1. LassoCV <br>
2. Permutation Feature Importance <br>

c. Correlation and outliers <br>
Highly correlated features will not be included. (Only few will be selected) <br>
Feature with too many outliers may lead a misleading result. Therefore these features may be filtered out